<a href="https://colab.research.google.com/github/ShotaSasaki-HCU/Prog2kakushin/blob/main/work2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Numpyの練習
# 2D Phased Array Simulator

import numpy as np

range_x_R = 1 # x軸方向レンジ右(mm)
range_x_L = -1 # x軸方向レンジ左(mm)
range_y_T = 1 # y軸方向レンジ上(mm)
range_y_B = -1 # y軸方向レンジ下(mm)
calc_width = 1 # 計算幅(mm)
tr_num = 4 # 送信素子数（個）
tr_directivity = 95 # 送信素子指向性（度）
tr_clearance = 8.4 # 送信素子間隔(mm)
tr_x_coord = [] # 送信素子のx座標（y座標は0で固定）
beam_direction = np.pi / 2 # 合成波の指向方向(rad)

# 送信素子をx軸上に原点が中心となる様に配置する。それらの座標を計算してtr_x_coordに格納する。
if (tr_num % 2) == 1: # 送信素子数が奇数個のとき
  for n in range(0, tr_num):
    tr_x_coord.append(-((tr_num // 2) * tr_clearance) + (n * tr_clearance))
else: # 送信素子数が偶数個のとき
  for n in range(0, tr_num):
    tr_x_coord.append(-(((tr_num // 2) * tr_clearance) - (tr_clearance / 2)) + (n * tr_clearance))

print(tr_x_coord)

[-12.600000000000001, -4.200000000000001, 4.199999999999999, 12.600000000000001]
